<H2> Preparando o ambiente</H2>

In [17]:
## Instala pacotes do requests
!pip install -q requests

## Instala Watermark
!pip install -q watermark

## Instala o psycopg2-binary -- para suprir dependência da lib sqlalchemy
!pip install psycopg2-binary -q


In [14]:
import sqlalchemy as sa
import requests
import pandas as pd

ModuleNotFoundError: No module named 'psycopg2'

In [3]:
#versões dos pacotes usados neste notebook
%reload_ext watermark
%watermark -a "Eduardo Zanghi" --iversions

Author: Eduardo Zanghi

sqlalchemy: 2.0.32
requests  : 2.32.3
pandas    : 1.5.1



- requests  : 2.32.3
- pandas    : 1.5.1
- sqlalchemy: 2.0.32

---

<H2> Configurando as conexão

In [4]:
# Definindo variáveis de conexão

## Fonte de dados
api_data = 'https://jsonplaceholder.typicode.com/todos/'

## Conexão BD
host = 'localhost'
port = '5432'
user = 'postgres'
password = 'Password123'
database = 'datalake'

In [19]:
# Criando conexão com BD
engine = sa.create_engine('postgresql://' + user + ":" + password + "@" + host + ":" + port + "/" + database)

In [22]:
# Obtendo os dados da API
response = requests.get(api_data)
data = response.json()
df = pd.DataFrame(data)

In [24]:
df.sample()

,userId,id,title,completed
21,2,22,distinctio vitae autem nihil ut molestias quo,True


In [25]:
# Verificando a quantidade de valores nulos no dataset.
df.isna().value_counts()

userId  id     title  completed
False   False  False  False        200
dtype: int64

---

In [111]:
## Experimentando em sqlite3

import sqlite3 as sql

In [110]:
# Criando a conexão a um banco de dados SQLite
cnn = sql.connect('dbproject1.db')

# Copia dados do Dataframe para dentro do banco de dados como uma tabela
df.to_sql('teddy_360', cnn, if_exists='replace', index=False)

# Carregamos a extensão SQL
%load_ext sql

# Defnindo o banco de dados SQLite
%sql sqlite:///dbproject1.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [114]:
%%sql

select * from teddy_360 limit 10

 * sqlite:///dbproject1.db
Done.


userId,id,title,completed
1,1,delectus aut autem,0
1,2,quis ut nam facilis et officia qui,0
1,3,fugiat veniam minus,0
1,4,et porro tempora,1
1,5,laboriosam mollitia et enim quasi adipisci quia provident illum,0
1,6,qui ullam ratione quibusdam voluptatem quia omnis,0
1,7,illo expedita consequatur quia in,0
1,8,quo adipisci enim quam ut ab,1
1,9,molestiae perspiciatis ipsa,0
1,10,illo est ratione doloremque quia maiores aut,1
